In [2]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [3]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [4]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [6]:
select
    med.NomeMedicamento,
    count(*) as qtd_pessoas
from medicamento as med
inner join
    UsoMedicamento as usoMed on usoMed.codMedicamento = med.codigo
group by med.NomeMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [8]:
DROP view if exists qtd_pessoas_por_med;

create view qtd_pessoas_por_med as (
select
    u.codMedicamento,
    count(*) as qtd_pessoas
from UsoMedicamento as u
group by u.codMedicamento
);

select
    *
from qtd_pessoas_por_med
order by qtd_pessoas desc
limit 1;

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [18]:
select
    u.codMedicamento,
    count(*) as qtd_pessoas
from UsoMedicamento as u
group by u.codMedicamento
order by 2 desc
limit 1

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [19]:
select
    med.NomeMedicamento,
    count(*) as qtd_pessoas
from medicamento as med
inner join
    UsoMedicamento as usoMed on usoMed.codMedicamento = med.codigo
group by med.NomeMedicamento
order by 2 desc
limit 1

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [13]:
select
    avg(qtd_med) as media_por_pessoa
from (
select
    count(*) as qtd_med
from UsoMedicamento
group by idPessoa
)

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [14]:
select
    idPessoa,
    count(*) as qtd_med
from UsoMedicamento
group by idPessoa
having count(*) > (select
    avg(qtd_med) as media_por_pessoa
from (
select
    count(*) as qtd_med
from UsoMedicamento
group by idPessoa
))

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [36]:
select
    idade,
    avg(qtd_med) as media_med
from (
    select
        p.idade,
        count(*) as qtd_med
    from pessoa as p
    inner join
        UsoMedicamento as u on u.idPessoa = p.id
    group by p.id
)
group by idade
order by 2 desc

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [25]:
drop view if exists coquetel;
drop view if exists medicamentoB;

create view medicamentoB as (
    select
        idPessoa,
        codMedicamento as medB
    from UsoMedicamento
);

create view coquetel as (
    select
        u.codMedicamento as medicamentoA,
        uB.medB as medicamentoB
    from UsoMedicamento as u
    inner join
        medicamentoB as uB on uB.idPessoa = u.idPessoa
    where u.codMedicamento > uB.medB
);

select * from coquetel;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [26]:
select
    medicamentoA,
    medicamentoB,
    count(*) as qtd_pessoas
from coquetel
group by medicamentoA, medicamentoB

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [27]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [29]:
update Matriz
set IdadeAte60 = true
where idade <= 60;

select * from Matriz where IdadeAte60

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [33]:
update Matriz
set MaisUmAnoMedicamento = true
where id in (select
                idPessoa
             from UsoMedicamento
             where diasuso > 365);
             
select * from Matriz where MaisUmAnoMedicamento

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [46]:
update Matriz
set MedicamentosAcimaMedia = true
where id in (select
                 idPessoa
             from (
                select
                    idPessoa,
                    count(*) as qtd_med
                from UsoMedicamento
                group by idPessoa
                having count(*) > (select
                    avg(qtd_med) as media_por_pessoa
                from (
                select
                    count(*) as qtd_med
                from UsoMedicamento
                group by idPessoa
                ))));
                
select * from Matriz where MedicamentosAcimaMedia

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Podemos analisar se o grupo de pessoas com idade menor que 60 anos possuí mais pessoas que usa um medicamento por mais de um ano do que o grupo de pessoas com mais de 60 anos. Ou seja, analisar qual grupo tem mais pessoas que tem um uso contínuo de um medicamento.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [47]:
select
    idadeAte60,
    count(*) qtd_pessoa_que_usam_med_por_mais_de_um_ano
from Matriz
where MaisUmAnoMedicamento
group by idadeAte60